In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.plscore_RMT_sigma import PLScoreRMT
from lib.models.LeeNet.score_RMT_sigma_center import ScorePureRMTCENTER
from lib.models.layer.RMT import PatchMerging
from lib.models.head.center_predictor import CenterPredictor
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PLScoreRMT(down_sample=PatchMerging, cfg=cfg)

    stride = cfg.model.backbone.stride
    feat_sz = int(cfg.data.search.size / stride)
    head = CenterPredictor(inplanes=cfg.model.pureRMT.embed_dim[-1], channel=cfg.model.head.num_channels, feat_sz=feat_sz, stride=stride)
    model = ScorePureRMTCENTER(backbone, head, cfg)
    return model


cfg = env_setting(cfg_name='plscore_pureRMT_sigma_center.yaml')
loader_train, loader_val = build_dataloaders(cfg)
data = None
# for i, _data in enumerate(loader_train, 1):
#     data = _data
#     break

/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# net = build_model(cfg)
# 
# focal_loss = FocalLoss()
# objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
# loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
# actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

In [4]:
# data = data.to(cfg.train.device)
# actor(data)

In [3]:
checkpoint_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/LeeNet_plScore_RMT_CENTER00001/ScorePureRMTCENTER_ep2600.pth.tar"
# checkpoint_dict = torch.load(checkpoint_path,map_location={"cuda:1":"cuda:1"})['net']
checkpoint_dict = torch.load(checkpoint_path)['net']

net = build_model(cfg)

model_dict = net.state_dict()

state_dict = {k: v for k, v in checkpoint_dict.items() if k in model_dict.keys()}
need_delete_keys = []
del state_dict['mlp.layers.0.weight']
for key in state_dict.keys():
    if 'backbone' in key or 'gen_feat_conv' in key or 'conv1_ctr' in key or 'conv1_offset' in key or 'conv1_size' in key:
        print("delete key: ", key)
        need_delete_keys.append(key)

print(need_delete_keys)


for key in need_delete_keys:
    del state_dict[key]
print(state_dict.keys())
model_dict.update(state_dict)
net.load_state_dict(model_dict, strict=False)

delete key:  backbone.patch_embed.proj.weight
delete key:  backbone.patch_embed.proj.bias
delete key:  backbone.score.conv1.conv.weight
delete key:  backbone.score.conv1.bn.weight
delete key:  backbone.score.conv1.bn.bias
delete key:  backbone.score.conv1.bn.running_mean
delete key:  backbone.score.conv1.bn.running_var
delete key:  backbone.score.conv1.bn.num_batches_tracked
delete key:  backbone.score.conv2.conv.weight
delete key:  backbone.score.conv2.bn.weight
delete key:  backbone.score.conv2.bn.bias
delete key:  backbone.score.conv2.bn.running_mean
delete key:  backbone.score.conv2.bn.running_var
delete key:  backbone.score.conv2.bn.num_batches_tracked
delete key:  backbone.score.confident_conv1.conv.weight
delete key:  backbone.score.confident_conv1.bn.weight
delete key:  backbone.score.confident_conv1.bn.bias
delete key:  backbone.score.confident_conv1.bn.running_mean
delete key:  backbone.score.confident_conv1.bn.running_var
delete key:  backbone.score.confident_conv1.bn.num_ba

<All keys matched successfully>

In [4]:
from collections import OrderedDict

RMT_L_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/RMT-L.pth"

RMT_L_dict = torch.load(RMT_L_path)['model']

RMT_state_dict = OrderedDict()
RMT_state_dict.clear()

for key, v in RMT_L_dict.items():
    if key.startswith("layers")  :
        key = key.replace("layers", "backbone.RMT_layers")
        RMT_state_dict[key] = v


model_dict = net.state_dict()
# print(model_dict)
state_dict = {k: v for k, v in RMT_state_dict.items() if k in model_dict.keys()}
for key in state_dict.keys():
    print(key)
model_dict.update(state_dict)
net.load_state_dict(model_dict, strict=False)

backbone.RMT_layers.0.Relpos.angle
backbone.RMT_layers.0.Relpos.decay
backbone.RMT_layers.0.blocks.0.retention_layer_norm.weight
backbone.RMT_layers.0.blocks.0.retention_layer_norm.bias
backbone.RMT_layers.0.blocks.0.retention.q_proj.weight
backbone.RMT_layers.0.blocks.0.retention.q_proj.bias
backbone.RMT_layers.0.blocks.0.retention.k_proj.weight
backbone.RMT_layers.0.blocks.0.retention.k_proj.bias
backbone.RMT_layers.0.blocks.0.retention.v_proj.weight
backbone.RMT_layers.0.blocks.0.retention.v_proj.bias
backbone.RMT_layers.0.blocks.0.retention.lepe.conv.weight
backbone.RMT_layers.0.blocks.0.retention.lepe.conv.bias
backbone.RMT_layers.0.blocks.0.retention.out_proj.weight
backbone.RMT_layers.0.blocks.0.retention.out_proj.bias
backbone.RMT_layers.0.blocks.0.final_layer_norm.weight
backbone.RMT_layers.0.blocks.0.final_layer_norm.bias
backbone.RMT_layers.0.blocks.0.ffn.fc1.weight
backbone.RMT_layers.0.blocks.0.ffn.fc1.bias
backbone.RMT_layers.0.blocks.0.ffn.fc2.weight
backbone.RMT_layers.

<All keys matched successfully>

In [5]:



focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

trainer.train(cfg.train.epoch)

Epoch Time: 0:00:28.901050
Avg Data Time: 24.91072
Avg GPU Trans Time: 0.14222
Avg Forward Time: 3.84811
Epoch Time: 0:00:31.529825
Avg Data Time: 12.50491
Avg GPU Trans Time: 0.14822
Avg Forward Time: 3.11178
Epoch Time: 0:00:34.113765
Avg Data Time: 8.37171
Avg GPU Trans Time: 0.14253
Avg Forward Time: 2.85702
Epoch Time: 0:00:36.684644
Avg Data Time: 6.29599
Avg GPU Trans Time: 0.14309
Avg Forward Time: 2.73209
Epoch Time: 0:00:39.303612
Avg Data Time: 5.05837
Avg GPU Trans Time: 0.14146
Avg Forward Time: 2.66089
Epoch Time: 0:00:41.907873
Avg Data Time: 4.22929
Avg GPU Trans Time: 0.14275
Avg Forward Time: 2.61260
Epoch Time: 0:00:44.534489
Avg Data Time: 3.64019
Avg GPU Trans Time: 0.14296
Avg Forward Time: 2.57893
Epoch Time: 0:00:47.183032
Avg Data Time: 3.20028
Avg GPU Trans Time: 0.14206
Avg Forward Time: 2.55554
Epoch Time: 0:00:49.789752
Avg Data Time: 2.85478
Avg GPU Trans Time: 0.13690
Avg Forward Time: 2.54052
[train: 1, 10 / 375] FPS: 30.5 (61.4)  ,  DataTime: 2.578 (0.1

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8861635dc0>
Traceback (most recent call last):
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/local/anaconda/envs/lzd-LeeNet/lib/python3.8/selectors.py", line 415, in select
 

Training crashed at epoch 2


KeyboardInterrupt: 